# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data= pd.read_csv("weather_data.csv")

weather_data.head()


,Cities,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Windspeed
0,boca do acre,59,BR,1586665308,91,-8.75,-67.40,23.38,0.31
1,tuy hoa,80,VN,1586665308,65,13.08,109.30,29.04,1.62
2,biltine,99,TD,1586665308,15,14.53,20.92,25.22,7.01
3,port alfred,37,ZA,1586665308,81,-33.59,26.89,18.33,2.24
4,pahasu,20,IN,1586665309,21,28.18,78.05,31.31,3.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
coordinates = weather_data[['Latitude','Longitude']]


weight = weather_data['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
import numpy as np
C_70 = (70-32)*5/9
C_80 = (80-32)*5/9

ideal_1 = weather_data[weather_data["Max Temperature"] > C_70]
ideal_2 = ideal_1[ideal_1["Max Temperature"] < C_80]
ideal_3 = ideal_2[ideal_2["Cloudiness"] == 0]
ideal_na = ideal_3.dropna()
ideal= ideal_na.reset_index()

ideal['Hotel'] = ""




for index, row in ideal.iterrows():
    

    params = {
        "location": f"{row['Latitude']}, {row['Longitude']}", 
        "keyword": "Hotel",
        "radius": 5000,
        "key": g_key,
    }
    #print(f"{row['Latitude']}, {row['Longitude']}")
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    try:
        ideal.loc[index, 'Hotel'] = results[0]['name']
        
    
    except:
        ideal.loc[index, 'Hotel'] = np.NaN

        
ideal
    


,index,Cities,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Windspeed,Hotel
0,54,san patricio,0,MX,1586665315,63,19.22,-104.70,24.11,5.29,Hotel Trivento
1,143,phek,0,IN,1586665326,21,25.67,94.50,24.51,0.68,HighWay Hotel
2,145,dwarka,0,IN,1586665326,76,22.24,68.97,26.61,5.62,"VITS Devbhumi, Dwarka"
3,154,kempsey,0,AU,1586665327,43,-31.08,152.83,22.78,0.45,PROVIDENCE ﻿onRIVER
4,159,tessalit,0,ML,1586665328,14,20.20,1.01,25.21,1.93,NaN
5,190,niquero,0,CU,1586665332,79,20.04,-77.58,25.43,1.98,Islazul Niquero
6,229,tecoanapa,0,MX,1586665338,82,16.52,-98.75,25.48,1.92,Hotel Garapacho
7,336,adrar,0,MR,1586665352,21,20.50,-10.07,24.46,4.52,NaN
8,394,cockburn town,0,TC,1586665358,79,21.46,-71.14,26.16,6.80,Osprey Beach Hotel
9,468,baracoa,0,CU,1586665364,81,20.35,-74.50,26.38,4.25,El Castillo


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
ideal_na= ideal.dropna()
ideal_na

,index,Cities,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Windspeed,Hotel
0,54,san patricio,0,MX,1586665315,63,19.22,-104.70,24.11,5.29,Hotel Trivento
1,143,phek,0,IN,1586665326,21,25.67,94.50,24.51,0.68,HighWay Hotel
2,145,dwarka,0,IN,1586665326,76,22.24,68.97,26.61,5.62,"VITS Devbhumi, Dwarka"
3,154,kempsey,0,AU,1586665327,43,-31.08,152.83,22.78,0.45,PROVIDENCE ﻿onRIVER
5,190,niquero,0,CU,1586665332,79,20.04,-77.58,25.43,1.98,Islazul Niquero
6,229,tecoanapa,0,MX,1586665338,82,16.52,-98.75,25.48,1.92,Hotel Garapacho
8,394,cockburn town,0,TC,1586665358,79,21.46,-71.14,26.16,6.80,Osprey Beach Hotel
9,468,baracoa,0,CU,1586665364,81,20.35,-74.50,26.38,4.25,El Castillo
10,520,port macquarie,0,AU,1586665374,35,-31.43,152.92,22.78,3.60,Sails Port Macquarie by Rydges
11,556,jaciara,0,BR,1586665379,82,-15.97,-54.97,22.14,0.53,Hotel Real


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
hotel_df = ideal_na
hotel_df

,index,Cities,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Windspeed,Hotel
0,54,san patricio,0,MX,1586665315,63,19.22,-104.70,24.11,5.29,Hotel Trivento
1,143,phek,0,IN,1586665326,21,25.67,94.50,24.51,0.68,HighWay Hotel
2,145,dwarka,0,IN,1586665326,76,22.24,68.97,26.61,5.62,"VITS Devbhumi, Dwarka"
3,154,kempsey,0,AU,1586665327,43,-31.08,152.83,22.78,0.45,PROVIDENCE ﻿onRIVER
5,190,niquero,0,CU,1586665332,79,20.04,-77.58,25.43,1.98,Islazul Niquero
6,229,tecoanapa,0,MX,1586665338,82,16.52,-98.75,25.48,1.92,Hotel Garapacho
8,394,cockburn town,0,TC,1586665358,79,21.46,-71.14,26.16,6.80,Osprey Beach Hotel
9,468,baracoa,0,CU,1586665364,81,20.35,-74.50,26.38,4.25,El Castillo
10,520,port macquarie,0,AU,1586665374,35,-31.43,152.92,22.78,3.60,Sails Port Macquarie by Rydges
11,556,jaciara,0,BR,1586665379,82,-15.97,-54.97,22.14,0.53,Hotel Real


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [72]:
import gmaps
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…